In [1]:
import findspark
findspark.init()
from pyspark import SparkContext as sc

import os

from pyspark.sql import SQLContext, Row

import pyspark

import sys
import string
from csv import reader
from functools import reduce
from pyspark.sql import functions as f
from collections import defaultdict
import datetime

sys.path.insert(0, '../src')
from tableCollections import TableCollections
import os

In [2]:
data_path = ['data']
parking_filepath = os.sep.join(data_path + ['parking.csv'])

sc = sc(appName="Test_all")
sqlContext = SQLContext(sc)

In [3]:
parkingTable = sqlContext.read.format('csv').options(header='true',inferschema='true').load('/user/ecc290/HW1data/parking-violations-header.csv')

In [4]:
openTable = sqlContext.read.format('csv').options(header='true',inferschema='true').load('/user/ecc290/HW1data/open-violations-header.csv')

In [5]:
!/usr/bin/hadoop fs -put '../data/us_cities_states_counties.csv'

In [6]:
# This file should be found on hdfs after the above hfs execution
category = sqlContext.read.format('csv').options(header='true',inferschema='true').load('us_cities_states_counties.csv')

In [7]:
tc = TableCollections(sqlContext, sc)

## Register the tables (create metadata if does not exists)

In [8]:
tc.register(parkingTable, "parking")

num metadata file exists for table parking
timestamp metadata file exists for table parking
string metadata file exists for table parking


In [9]:
tc.register(openTable, "open")

num metadata file exists for table open
string metadata file exists for table open


In [10]:
# This line must be run in order to run some methods
tc.register(category, "category")

## Returns names of time columns within a range of dates 

In [11]:
import datetime
tc.timeColWithinRange(datetime.datetime(1994,1,1), datetime.datetime(2018,5,1)).show()

+----------+---------+
|   colName|tableName|
+----------+---------+
|issue_date|  parking|
+----------+---------+



## Returns name of Numerical columns within a range of values

In [12]:
tc.numColWithinRange(-1000000, 10000000).show()

+------------------+---------+
|           colName|tableName|
+------------------+---------+
|violation_location|  parking|
|violation_precinct|  parking|
|   issuer_precinct|  parking|
|       issuer_code|  parking|
|    violation_code|  parking|
|      vehicle_year|  parking|
|  reduction_amount|     open|
|   interest_amount|     open|
|    payment_amount|     open|
|    penalty_amount|     open|
|       fine_amount|     open|
|        amount_due|     open|
|          precinct|     open|
+------------------+---------+



## Returns range of value(minimum and maximum) that a column has in particular table

In [13]:
colList= ["parking^violation_location","parking^violation_precinct", "open^reduction_amount", "open^amount_due"]
tc.getNumRange(colList).show()

+---------+------------------+----+-----+
|tableName|           colName| min|  max|
+---------+------------------+----+-----+
|  parking|violation_location|-1.0|967.0|
|  parking|violation_precinct| 0.0|967.0|
|     open|  reduction_amount| 0.0|545.0|
|     open|        amount_due| 0.0|594.1|
+---------+------------------+----+-----+



## Returns time range of timestame columns

In [14]:
colList= ["parking^issue_date"]
tc.getTimeRange(colList).show()

+---------+----------+-------------------+-------------------+
|tableName|   colName|                min|                max|
+---------+----------+-------------------+-------------------+
|  parking|issue_date|2016-03-01 00:00:00|2016-03-31 00:00:00|
+---------+----------+-------------------+-------------------+



## Return column names having intersection over union greater than a threshold

In [15]:
tc.getSimilarNumCols("parking^violation_location", threshold=0.1).show()

+------------------+---------+------------------+
|           colName|tableName|               iou|
+------------------+---------+------------------+
|violation_location|  parking|               1.0|
|violation_precinct|  parking|0.9989669421487604|
|   issuer_precinct|  parking| 0.973816717019134|
|          precinct|     open|0.8316115702479339|
|        amount_due|     open|0.6137396694214876|
|    payment_amount|     open|0.6099586776859505|
|  reduction_amount|     open|0.5630165289256198|
|       fine_amount|     open| 0.506198347107438|
|      vehicle_year|  parking|0.4671497584541063|
|    violation_code|  parking|0.1012396694214876|
+------------------+---------+------------------+



## Returns intersecting values in two columns of different or same tables

In [16]:
colList= ["parking^plate_id","open^plate"]
tc.returnIntersecWithinCols(colList).show()

+---------+-----+
|col_value|count|
+---------+-----+
|  ACL4445|    2|
|  GGJ4899|    2|
|  HBH9968|    2|
|  GPL7023|    2|
|  BVS5686|    2|
|  HDU4451|    2|
|  HAY6630|    2|
|  20213MA|    2|
|  GDN2939|    2|
|  HDZ8438|    2|
|  55411JS|    2|
|  CFJ8401|    2|
|  44366MD|    2|
|  VFR2928|    2|
|   XCSB43|    2|
|  DSZ6353|    2|
|  GKL8441|    2|
| T649254C|    2|
|  VBU9385|    2|
|  GKN1233|    2|
+---------+-----+
only showing top 20 rows



## Return the top n most frequent values in different tables and columns

In [17]:
colList= ["parking^plate_id","open^plate"]
tc.frequentVals(colList, 5).show()

+----------+----+--------+
| col_value| cnt|col_name|
+----------+----+--------+
|BLANKPLATE|1203|plate_id|
|BLANKPLATE| 872|   plate|
|       N/A| 155|plate_id|
|       N/A| 149|   plate|
|    AP501F| 138|plate_id|
+----------+----+--------+



## Return outlier based on frequency beyond a theshold in a column in specified table

In [18]:
colList= ["parking^plate_id","open^plate"]
tc.returnOutliers(colList, 0.01)

+---------+---+--------+
|col_value|cnt|col_name|
+---------+---+--------+
|  BML4037|  1|plate_id|
| T655130C|  1|plate_id|
| T649453C|  1|plate_id|
|  CHU7059|  1|plate_id|
|   D57CVL|  1|plate_id|
|  EWX5634|  1|plate_id|
|  30437MA|  1|plate_id|
|  DUA2419|  1|plate_id|
|   Z77FJX|  1|plate_id|
|  ERG9119|  1|plate_id|
|  EWD7040|  1|plate_id|
|  FYW3414|  1|plate_id|
|  DRV8081|  1|plate_id|
|  FRJ7147|  1|plate_id|
|  39034MG|  1|plate_id|
|   Z41DYN|  1|plate_id|
|   N77FNS|  1|plate_id|
|   E91EYH|  1|plate_id|
|  CPZ3300|  1|plate_id|
|  EHD5439|  1|plate_id|
+---------+---+--------+
only showing top 20 rows

+---------+---+--------+
|col_value|cnt|col_name|
+---------+---+--------+
|  GWM2129|  1|   plate|
|  GVS9200|  1|   plate|
|  ACL4445|  1|   plate|
|  GGJ4899|  1|   plate|
|  HBH9968|  1|   plate|
|   595ZSY|  1|   plate|
|  BVS5686|  1|   plate|
|  HDU4451|  1|   plate|
|  20213MA|  1|   plate|
|  EBE1052|  1|   plate|
|  GDN2939|  1|   plate|
|  GFF4794|  1|   plate|

## Column names in a table with and without some keywords

In [19]:
colList= ["parking^plate_id","open^plate"]
withList = ['CPZ3300']
withoutList = ['GTR366A']
tc.colsWithAndWithout(colList, withList, withoutList).show()

+--------+----------+
|col_name|table_name|
+--------+----------+
|   plate|      open|
+--------+----------+



## Finds the number of unique values in a columns in specified table

In [20]:
tc.getCardinality(colList).show()

+--------+------+
|col_name| count|
+--------+------+
|   plate|212692|
|plate_id|607520|
+--------+------+



## Returns columns of specified datatype in a dataframe

In [21]:
tc.getColsOfDatatype(openTable, 'string')

['plate',
 'license_type',
 'county',
 'state',
 'issuing_agency',
 'violation',
 'violation_status',
 'issue_date',
 'violation_time',
 'judgment_entry_date']

In [22]:
#Necessary to use nltk library
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jp4989/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Returns similarity measures for columns having path similarity greater than 0.5 for different dataframe or a keyword category

In [23]:
tc.colsNameSimilarity(openTable, category = None, df2=parkingTable).show()

+----------------+--------------------+---------------+--------------------+----------------+
|        Column_1|            Column_2|Path Similarity|levenshtein distance|soundex_equality|
+----------------+--------------------+---------------+--------------------+----------------+
|  summons_number|      summons_number|            1.0|                   0|            true|
|  summons_number|        meter_number|            1.0|                   6|           false|
|           plate|            plate_id|            1.0|                   3|           false|
|           plate|          plate_type|            1.0|                   5|           false|
|    license_type|          plate_type|            1.0|                   6|           false|
|    license_type|   vehicle_body_type|            1.0|                   9|           false|
|          county|    violation_county|            1.0|                  10|           false|
|           state|  registration_state|            1.0|     

In [24]:
tc.colsNameSimilarity(openTable, category = 'state', df2=None).show()

+--------+---------------+--------------------+----------------+
|Column_1|Path Similarity|levenshtein distance|soundex_equality|
+--------+---------------+--------------------+----------------+
|   state|            1.0|                   0|            true|
+--------+---------------+--------------------+----------------+



In [25]:
tc.colsNameSimilarity(parkingTable, category = 'state', df2=None).show()

+------------------+---------------+--------------------+----------------+
|          Column_1|Path Similarity|levenshtein distance|soundex_equality|
+------------------+---------------+--------------------+----------------+
|registration_state|            1.0|                  13|           false|
+------------------+---------------+--------------------+----------------+



## Find if a column is of type State(full, short), County and City for United States

In [26]:
tc.getColsofCategory('parking',['registration_state'],category= 'State_full').show()

Column values are not a subset of State_full
+---------+------------------+----------+--------+
|tableName|           colName|  category|IsSubset|
+---------+------------------+----------+--------+
|  parking|registration_state|State_full|   false|
+---------+------------------+----------+--------+



In [27]:
tc.getColsofCategory('parking',['registration_state'],category= 'State_short').show()

Column values are not a subset of State_short
+---------+------------------+-----------+--------+
|tableName|           colName|   category|IsSubset|
+---------+------------------+-----------+--------+
|  parking|registration_state|State_short|   false|
+---------+------------------+-----------+--------+



## Returns number of null values in specified table and columns

In [28]:
tc.countNullValues([['parking', 'violation_county']]).show()

+----------+----------------+-----------+
|Table name|     column name|Null Values|
+----------+----------------+-----------+
|   parking|violation_county|   192974.0|
+----------+----------------+-----------+



## Return Union values in two columns of same or different tables

In [29]:
colList= ["parking^plate_id","open^plate"]
tc.returnUnionWithinCols(colList).show()

+---------+-----+
|col_value|count|
+---------+-----+
|  ACL4445|    2|
|  GGJ4899|    2|
|  HBH9968|    2|
|  GPL7023|    2|
|  BVS5686|    2|
|  HDU4451|    2|
|  HAY6630|    2|
|  20213MA|    2|
|  GDN2939|    2|
|  HDZ8438|    2|
|  55411JS|    2|
|  CFJ8401|    2|
|  44366MD|    2|
|  VFR2928|    2|
|   XCSB43|    2|
|  DSZ6353|    2|
|  GKL8441|    2|
| T649254C|    2|
|  VBU9385|    2|
|  GKN1233|    2|
+---------+-----+
only showing top 20 rows

